In [1]:
import os
import datetime
import warnings
import polars as pl
import pandas as pd
import altair as alt

from src.najdi_rok import najdi_rok
from src.pocet_stran import pocet_stran
from src.bez_bordelu import bez_bordelu
from src.alt_friendly import alt_friendly
from src.hezke_jmeno import hezke_jmeno
from src.kristi_promin import kristi_promin
from src.zjisti_vazbu import zjisti_vazbu

pl.Config(tbl_rows=1000)
alt.data_transformers.disable_max_rows()
alt.themes.register('irozhlas', kristi_promin)
alt.themes.enable('irozhlas')
warnings.filterwarnings('ignore')

In [2]:
df = pl.read_parquet(os.path.join("data/cnb_sloupce","100.parquet"))
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","leader.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","008.parquet")), left_on="001", right_on="001", how="left")
df = df.to_pandas()
df = df[df["leader"].str[6].isin(["a", "t"])]
df = df[~df["leader"].str[7].isin(["b", "i", "s", " "])]
df = df[(df["008"].str[15:17] == "xr") & (df["008"].str[35:38] == "cze")]
df = pl.from_pandas(df)
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","020.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","022.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","245.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","300.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","650.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","653.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","655.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","700.parquet")), left_on="001", right_on="001", how="left")
df = df.explode("022_a").filter(pl.col("022_a").is_null())
df = df.with_columns(pl.col('008').map_elements(najdi_rok, return_dtype=int).alias('rok'))
df = df.with_columns(pl.col('300_a').map_elements(pocet_stran, return_dtype=int).alias('stran'))
df = df.with_columns(pl.col('245_a').map_elements(bez_bordelu, return_dtype=str))
df = df.explode("020_q").with_columns(pl.col("020_q").map_elements(zjisti_vazbu, return_dtype=str).alias('vazba'))
df = df.explode('245_p').with_columns(pl.col('245_p').map_elements(bez_bordelu, return_dtype=str))
print(len(df))
df = df.filter(pl.col("stran") >= 30)

df = df.filter((~pl.col("245_h").str.contains("grafika")) | pl.col("245_h").is_null()).unique(subset=["100_a","245_a","245_p"], keep="first")
print(len(df))

795736
506565


## Kdy se vyplnovalo pole 655_a?

In [4]:
df.filter(~pl.col("655_a").is_null()).group_by("rok").len().sort(by="rok").join(
    df.group_by('rok').len(), on='rok', how='left'
).with_columns((pl.col("len") / pl.col("len_right")).alias('vyplnenost')).tail(50)

rok,len,len_right,vyplnenost
i64,u32,u32,f64
1976,623,2825,0.220531
1977,669,2892,0.231328
1978,710,2935,0.241908
1979,901,3218,0.279988
1980,1037,3321,0.312255
1981,1036,3164,0.327434
1982,1449,3380,0.428698
1983,2483,3884,0.639289
1984,2523,3989,0.632489


In [5]:
df = df.filter(pl.col("rok").is_between(1989,2024))

In [6]:
df.filter(pl.col("245_a").str.contains("Každý den je nový")).explode("655_a")

100_ind1,100_a,100_7,100_4,100_d,100_q,100_c,100_b,100_e,001,leader,008,020_q,020_c,020_a,020_z,022_a,022_y,022_z,022_ind1,022_l,245_ind1,245_ind2,245_a,245_b,245_c,245_n,245_p,245_h,245_f,245_s,300_a,300_b,300_c,300_e,300_f,300_3,…,653_a,655_ind2,655_a,655_7,655_2,655_ind1,655_y,655_z,700_ind1,700_a,700_4,700_d,700_7,700_t,700_q,700_l,700_ind2,700_c,700_b,700_i,700_m,700_k,700_n,700_r,700_p,700_o,700_s,700_j,700_x,700_e,700_f,700_5,700_9,700_g,rok,stran,vazba
str,str,str,list[str],str,str,list[str],str,str,str,str,str,str,list[str],list[str],list[str],str,list[str],list[str],str,str,str,str,str,str,str,list[str],str,str,str,str,list[str],list[str],list[str],list[str],str,str,…,list[str],list[str],str,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],i64,i64,str
"""1""","""Lomová, Lucie,""","""xx0026705""","[""aut"", ""ill""]","""1964-""",null,null,null,null,"""nkc20223471706""",""" nam a22 i 4500""","""221110s2022 xr a g 6 0…","""(vázáno)""",null,"[""978-80-88378-17-4""]",null,null,null,null,null,null,"""1""","""0""","""Každý den je nový""","""komiksový deník /""","""Lucie Lomová""",null,null,null,null,null,"[""106, 98 stran :""]","[""barevné ilustrace ;""]","[""21 cm""]",null,null,null,…,null,"[""7"", ""7"", … ""9""]","""autobiografické komiksy""","[""fd1058616"", ""fd132006"", … null]","[""czenas"", ""czenas"", … ""eczenas""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2022,106,"""pevná"""
"""1""","""Lomová, Lucie,""","""xx0026705""","[""aut"", ""ill""]","""1964-""",null,null,null,null,"""nkc20223471706""",""" nam a22 i 4500""","""221110s2022 xr a g 6 0…","""(vázáno)""",null,"[""978-80-88378-17-4""]",null,null,null,null,null,null,"""1""","""0""","""Každý den je nový""","""komiksový deník /""","""Lucie Lomová""",null,null,null,null,null,"[""106, 98 stran :""]","[""barevné ilustrace ;""]","[""21 cm""]",null,null,null,…,null,"[""7"", ""7"", … ""9""]","""deníky""","[""fd1058616"", ""fd132006"", … null]","[""czenas"", ""czenas"", … ""eczenas""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2022,106,"""pevná"""
"""1""","""Lomová, Lucie,""","""xx0026705""","[""aut"", ""ill""]","""1964-""",null,null,null,null,"""nkc20223471706""",""" nam a22 i 4500""","""221110s2022 xr a g 6 0…","""(vázáno)""",null,"[""978-80-88378-17-4""]",null,null,null,null,null,null,"""1""","""0""","""Každý den je nový""","""komiksový deník /""","""Lucie Lomová""",null,null,null,null,null,"[""106, 98 stran :""]","[""barevné ilustrace ;""]","[""21 cm""]",null,null,null,…,null,"[""7"", ""7"", … ""9""]","""autobiographical comics""","[""fd1058616"", ""fd132006"", … null]","[""czenas"", ""czenas"", … ""eczenas""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2022,106,"""pevná"""
"""1""","""Lomová, Lucie,""","""xx0026705""","[""aut"", ""ill""]","""1964-""",null,null,null,null,"""nkc20223471706""",""" nam a22 i 4500""","""221110s2022 xr a g 6 0…","""(vázáno)""",null,"[""978-80-88378-17-4""]",null,null,null,null,null,null,"""1""","""0""","""Každý den je nový""","""komiksový deník /""","""Lucie Lomová""",null,null,null,null,null,"[""106, 98 stran :""]","[""barevné ilustrace ;""]","[""21 cm""]",null,null,null,…,null,"[""7"", ""7"", … ""9""]","""diaries""","[""fd1058616"", ""fd132006"", … null]","[""czenas"", ""czenas"", … ""eczenas""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2022,106,"""pevná"""
"""1""","""Mandžukov

In [7]:
df.filter(pl.col("245_a").str.contains("Pod dekou"))

100_ind1,100_a,100_7,100_4,100_d,100_q,100_c,100_b,100_e,001,leader,008,020_q,020_c,020_a,020_z,022_a,022_y,022_z,022_ind1,022_l,245_ind1,245_ind2,245_a,245_b,245_c,245_n,245_p,245_h,245_f,245_s,300_a,300_b,300_c,300_e,300_f,300_3,…,653_a,655_ind2,655_a,655_7,655_2,655_ind1,655_y,655_z,700_ind1,700_a,700_4,700_d,700_7,700_t,700_q,700_l,700_ind2,700_c,700_b,700_i,700_m,700_k,700_n,700_r,700_p,700_o,700_s,700_j,700_x,700_e,700_f,700_5,700_9,700_g,rok,stran,vazba
str,str,str,list[str],str,str,list[str],str,str,str,str,str,str,list[str],list[str],list[str],str,list[str],list[str],str,str,str,str,str,str,str,list[str],str,str,str,str,list[str],list[str],list[str],list[str],str,str,…,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],i64,i64,str
"""1""","""Thompson, Craig,""","""xx0037934""","[""aut"", ""ill""]","""1975-""",null,null,null,null,"""nkc20051632284""",""" nam a22 a 4500""","""051123s2005 xr a g 0…","""(váz.) :""","[""Kč 590,00""]","[""80-7341-603-4""]",null,null,null,null,null,null,"""1""","""0""","""Pod dekou""","""ilustrovaný román /""","""Craig Thompson ; [z anglického…",null,null,null,null,null,"[""582 s. :""]","[""il. ;""]","[""25 cm""]",null,null,null,…,null,"[""7"", ""7"", … ""9""]","[""americké romány"", ""komiksy"", … ""comics""]","[""fd131796"", ""fd131978"", … null]","[""czenas"", ""czenas"", … ""eczenas""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2005,582,"""pevná"""


In [8]:
zebricek = df.explode("655_a").group_by("655_a").len().sort(by="len",descending=True)
zebricek.filter(pl.col('len') >= 200)

655_a,len
str,u32
"""příručky""",32562
"""handbooks and manuals""",24458
"""učebnice vysokých škol""",21487
"""publikace pro děti""",19853
"""monografie""",18122
"""children's literature""",16902
"""monographs""",16491
"""textbooks (higher)""",13582
"""populárně-naučné publikace""",13514


In [9]:
zebricek.filter(pl.col("655_a").str.contains("dívč"))

655_a,len
str,u32
"""dívčí romány""",210


In [10]:
df.sample(20)

100_ind1,100_a,100_7,100_4,100_d,100_q,100_c,100_b,100_e,001,leader,008,020_q,020_c,020_a,020_z,022_a,022_y,022_z,022_ind1,022_l,245_ind1,245_ind2,245_a,245_b,245_c,245_n,245_p,245_h,245_f,245_s,300_a,300_b,300_c,300_e,300_f,300_3,…,653_a,655_ind2,655_a,655_7,655_2,655_ind1,655_y,655_z,700_ind1,700_a,700_4,700_d,700_7,700_t,700_q,700_l,700_ind2,700_c,700_b,700_i,700_m,700_k,700_n,700_r,700_p,700_o,700_s,700_j,700_x,700_e,700_f,700_5,700_9,700_g,rok,stran,vazba
str,str,str,list[str],str,str,list[str],str,str,str,str,str,str,list[str],list[str],list[str],str,list[str],list[str],str,str,str,str,str,str,str,list[str],str,str,str,str,list[str],list[str],list[str],list[str],str,str,…,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],i64,i64,str
"""1""","""Chmelík, Jan,""","""jn20000403070""","[""aut""]","""1952-""",null,null,null,null,"""nkc20051567822""",""" nam a22 a 4500""","""050519s2005 xr a e p 0…","""(brož.)""",null,"[""80-7201-543-5""]",null,null,null,null,null,null,"""1""","""0""","""Ekologická kriminalita a možno…","""vysokoškolská učebnice /""","""Jan Chmelík a kolektiv""",null,null,null,null,null,"[""215 s. :""]","[""il. (některé barev.) ;""]","[""21 cm""]",null,null,null,…,null,"[""7"", ""9""]","[""učebnice vysokých škol"", ""textbooks (higher)""]","[""fd133772"", null]","[""czenas"", ""eczenas""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2005,215,"""brožovaná"""
"""1""","""Kopecká, Věra,""","""jn99240000488""","[""aut"", ""pbl""]","""1951-""",null,null,null,null,"""nkc20162787573""",""" nam a22 i 4500""","""160516s2016 xr ac g 0…","""(brožováno)""",null,"[""978-80-88040-13-2""]",null,null,null,null,null,null,"""1""","""0""","""Když Broumovské stěny přetékaj…",null,"""Věra Kopecká a kolektiv""",null,null,null,null,null,"[""44 stran :""]","[""ilustrace (převážně barevné), portréty ;""]","[""30 cm""]",null,null,null,…,null,"[""7"", ""7"", … ""9""]","[""česká poezie"", ""polská poezie"", … ""collections of works""]","[""fd133958"", ""fd133080"", … null]","[""czenas"", ""czenas"", … ""eczenas""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2016,44,"""brožovaná"""
"""1""","""Hart, Emilia""","""xx0315431""","[""aut""]",null,null,null,null,null,"""nkc20243624288""",""" nam a22 i 4500""","""240725s2024 xr g 0…","""(vázáno) :""","[""Kč 399,00""]","[""978-80-277-2252-5""]",null,null,null,null,null,null,"""1""","""0""","""Divé ženy""",null,"""Emilia Hart ; přeložila Nika E…",null,null,null,null,null,"[""359 stran ;""]",null,"[""21 cm""]",null,null,null,…,null,"[""7"", ""7"", … ""9""]","[""australské romány"", ""historické romány"", … ""historical novels""]","[""fd131842"", ""fd132414"", … null]","[""czenas"", ""czenas"", … ""eczenas""]",null,null,null,"[""1""]","[""Exnerová, Nika,""]","[""trl""]","[""1989-""]","[""hka2016935782""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2024,359,"""pevná"""
"""1""","""Ragab, Saad Alamin Mohamed,""","""mzk2003195707""","[""aut""]","""1968-""",null,null,null,null,"""cpk20031254383""",""" cam a22 a 4500""","""030804s2003 xr a e 0…","""(brož.)""",null,"[""80-214-2418-4""]",null,null,null,null,null,null,"""1""","""0""","""Příspěvek k analýze dynamickýc…","""Analyze of dynamical state of …","""Saad Alamin Mohamed Ragab""",null,null,null,null,null,"[""38 s. :""]","[""il. ;""]","[""21 cm""]",null,null,null,…,null,"[""7"", ""9""]","[""teze"", ""abstracts""]","[""fd133701"", null]","[""czenas"", ""eczenas""]",null,null,null,null,null,null,null,null,null,nul

In [11]:
vyvoj = df.filter(pl.col("rok").is_between(2000,2003)).explode("655_a").group_by("655_a").len().join(
    df.filter(pl.col("rok").is_between(2021,2024)).explode("655_a").group_by("655_a").len(), on="655_a", how="full"
).with_columns((pl.col("len") / pl.col("len_right")).alias("rozdil")).with_columns((pl.col("len") + pl.col("len_right")).alias("celkem")).sort(by="rozdil")

In [12]:
vyvoj.filter(pl.col("celkem") >= 100).head(150)

655_a,len,655_a_right,len_right,rozdil,celkem
str,u32,str,u32,f64,u32
"""women's novels""",1,"""women's novels""",596,0.001678,597
"""romány pro ženy""",1,"""romány pro ženy""",596,0.001678,597
"""fantasy comics""",1,"""fantasy comics""",259,0.003861,260
"""fantasy komiksy""",1,"""fantasy komiksy""",259,0.003861,260
"""akční a dobrodružné komiksy""",1,"""akční a dobrodružné komiksy""",255,0.003922,256
"""action and adventure comics""",1,"""action and adventure comics""",252,0.003968,253
"""superhrdinské komiksy""",1,"""superhrdinské komiksy""",221,0.004525,222
"""superhero comics""",1,"""superhero comics""",219,0.004566,220
"""autorské knihy""",1,"""autorské knihy""",165,0.006061,166


In [13]:
vyvoj.filter(pl.col("celkem") >= 100).tail(150)

655_a,len,655_a_right,len_right,rozdil,celkem
str,u32,str,u32,f64,u32
"""Czech prose""",147,"""Czech prose""",429,0.342657,576
"""detective stories""",27,"""detective stories""",78,0.346154,105
"""obrazové publikace""",372,"""obrazové publikace""",1064,0.349624,1436
"""české romány""",751,"""české romány""",2097,0.358131,2848
"""texts""",38,"""texts""",101,0.376238,139
"""catalogs""",66,"""catalogs""",175,0.377143,241
"""popular works""",742,"""popular works""",1966,0.377416,2708
"""fotografické publikace""",556,"""fotografické publikace""",1459,0.381083,2015
"""autobiografické prózy""",44,"""autobiografické prózy""",114,0.385965,158


In [14]:
df.select(pl.col("653_a")).drop_nulls().sample(50)

653_a
list[str]
"[""emise cenných papírů""]"
"[""Beallovy seznamy""]"
"[""technika kumihimo""]"
"[""reformní socialismus""]"
"[""denní obchodování""]"
"[""Evropská unie"", ""dělba pravomocí"", … ""společný trh""]"
"[""omalovánky"", ""veršované pohádky"", ""české pohádky""]"
"[""kamevéda""]"
"[""homogenita teplotní řady"", ""meteorologické observatoře"", … ""teplotní řada""]"


In [15]:
vyvoj2 = df.filter(pl.col("rok").is_between(2000,2003)).explode("653_a").group_by("653_a").len().join(
    df.filter(pl.col("rok").is_between(2021,2024)).explode("653_a").group_by("653_a").len(), on="653_a", how="full"
).with_columns((pl.col("len") / pl.col("len_right")).alias("rozdil")).fill_null(0).with_columns((pl.col("len") + pl.col("len_right")).alias("celkem")).sort(by="rozdil")
vyvoj2.filter(pl.col("celkem") > 5)

653_a,len,653_a_right,len_right,rozdil,celkem
str,u32,str,u32,f64,u32
null,0,"""jazyková úroveň A1""",15,0.0,15
null,0,"""jazyková úroveň B2""",7,0.0,7
null,0,"""jazyková úroveň A2""",14,0.0,14
null,0,"""literatura new adult""",50,0.0,50
null,0,"""jazyková úroveň B1""",14,0.0,14
null,0,null,32189,0.0,32189
null,31258,null,0,0.0,31258
"""denní četba""",13,"""denní četba""",44,0.295455,57


In [16]:
vyvoj3 = df.filter(pl.col("rok").is_between(2000,2003)).explode("655_a").group_by("655_a").len().join(
    df.filter(pl.col("rok").is_between(2021,2024)).explode("655_a").group_by("655_a").len(), on="655_a", how="full"
).with_columns((pl.col("len") / pl.col("len_right")).alias("rozdil")).fill_null(0).with_columns((pl.col("len") + pl.col("len_right")).alias("celkem")).sort(by="rozdil")

In [17]:
vyvoj3.filter((pl.col("len") > 1) & (pl.col('len_right') > 1) & (pl.col("celkem") > 100)).sort(by='rozdil',descending=True)

655_a,len,655_a_right,len_right,rozdil,celkem
str,u32,str,u32,f64,u32
"""povídky""",112,"""povídky""",6,18.666667,118
"""milostné povídky""",492,"""milostné povídky""",37,13.297297,529
"""lyrická poezie""",187,"""lyrická poezie""",20,9.35,207
"""lyric poetry""",148,"""lyric poetry""",20,7.4,168
"""právní předpisy""",225,"""právní předpisy""",32,7.03125,257
"""učebnice vysokých škol""",3224,"""učebnice vysokých škol""",536,6.014925,3760
"""legal regulations""",153,"""legal regulations""",29,5.275862,182
"""studie""",2336,"""studie""",484,4.826446,2820
"""vědecko-fantastické povídky""",229,"""vědecko-fantastické povídky""",50,4.58,279


In [18]:
vyvoj3.filter(pl.col("celkem") > 200).filter(pl.col('rozdil') > 1).sort(by='rozdil',descending=True)

655_a,len,655_a_right,len_right,rozdil,celkem
str,u32,str,u32,f64,u32
"""milostné povídky""",492,"""milostné povídky""",37,13.297297,529
"""lyrická poezie""",187,"""lyrická poezie""",20,9.35,207
"""právní předpisy""",225,"""právní předpisy""",32,7.03125,257
"""učebnice vysokých škol""",3224,"""učebnice vysokých škol""",536,6.014925,3760
"""studie""",2336,"""studie""",484,4.826446,2820
"""vědecko-fantastické povídky""",229,"""vědecko-fantastické povídky""",50,4.58,279
"""textbooks (higher)""",2249,"""textbooks (higher)""",533,4.219512,2782
"""učebnice""",392,"""učebnice""",113,3.469027,505
"""dobrodružné povídky""",315,"""dobrodružné povídky""",101,3.118812,416


In [260]:
top = ['komiksy','young adult',"erotické romány", "rozhovory"]
flop = ["učebnice","slovníky","právní předpisy","encyklopedie"]

In [200]:
def podil_zanru(zanr, rok=1989):
    return df.filter(pl.col("rok") >= rok).explode("655_a").filter(
        pl.col("655_a").str.contains("(?i)" + zanr)
    ).group_by(
        "rok"
    ).len(
    ).join(df.filter(pl.col("rok") >= rok).drop_nulls(subset=["655_a"]).group_by("rok").len(), on="rok", how="right").with_columns((pl.col("len") / pl.col("len_right")).alias("podil")).sort(by="rok").rename({'len':'pocet'}).with_columns(pl.lit(zanr).alias("zanr"))

In [21]:
df.explode("655_a").filter(pl.col('655_a') == "erotické romány").group_by('245_a').len().sort(by='len',descending=True)

245_a,len
str,u32
"""Barvy lásky""",4
"""Hříšné odstíny vášně""",4
"""Jezdci apokalypsy""",4
"""Vlci zvěrokruhu""",4
"""Královská sága plná sexu""",4
"""Twisted""",4
"""Hot Jocks""",3
"""Poslední jezdci""",3
"""Tajemný hříšník""",3


In [22]:
def co_to_je(nazev):
    return df.filter(pl.col("245_a").str.contains(nazev)).select("655_a").explode("655_a").to_series().to_list()

In [196]:
co_to_je("Neztratit se sama sobě")

['rozhovory', 'interviews']

In [24]:
co_to_je("Otevřené hranice")

['naučné komiksy', 'nonfiction comics']

In [272]:
podil_zanru("nonfiction comics")

pocet,rok,len_right,podil,zanr
u32,i64,u32,f64,str
null,1989,3115,null,"""nonfiction comics"""
null,1990,3220,null,"""nonfiction comics"""
null,1991,3598,null,"""nonfiction comics"""
null,1992,4452,null,"""nonfiction comics"""
null,1993,5949,null,"""nonfiction comics"""
null,1994,6067,null,"""nonfiction comics"""
null,1995,6784,null,"""nonfiction comics"""
null,1996,7014,null,"""nonfiction comics"""
null,1997,7395,null,"""nonfiction comics"""


In [274]:
podil_zanru("komiks")

pocet,rok,len_right,podil,zanr
u32,i64,u32,f64,str
4,1989,3115,0.001284,"""komiks"""
6,1990,3220,0.001863,"""komiks"""
8,1991,3598,0.002223,"""komiks"""
8,1992,4452,0.001797,"""komiks"""
5,1993,5949,0.00084,"""komiks"""
11,1994,6067,0.001813,"""komiks"""
8,1995,6784,0.001179,"""komiks"""
6,1996,7014,0.000855,"""komiks"""
10,1997,7395,0.001352,"""komiks"""


In [262]:
top_data = pl.concat([podil_zanru(z, rok=2000) for z in top])
flop_data = pl.concat([podil_zanru(z, rok=2000) for z in flop])

In [242]:
# flop_data = flop_data.with_columns(pl.col("zanr").replace({"právní předpisy":"práv. předp.","encyklopedie":"encykl."}))

In [270]:
top_graf = alt.Chart(
    alt_friendly(top_data),title="Čeho vychází víc než v roce 2000…", width=55, height=55
).mark_area().encode(
    alt.X("rok:T", title=None),
    alt.Y("pocet", title=None, axis=alt.Axis(orient='right', domainOpacity=0, tickColor='#DCDDD6'), ),
    alt.Column("zanr", title=None, sort=top,
           header=alt.Header(labelFont='Asap'))
).resolve_axis(y="independent",x="independent").resolve_scale(y="independent")

flop_graf = alt.Chart(
    alt_friendly(flop_data),title="…a čeho míň", width=48, height=55
).mark_area().encode(
    alt.X("rok:T", title=None),
    alt.Y("pocet", title=None, axis=alt.Axis(orient='right', domainOpacity=0, tickColor='#DCDDD6')),
    alt.Column("zanr", title=None,
           header=alt.Header(labelFont='Asap'), sort=flop)
).resolve_axis(y="independent",x="independent").resolve_scale(y="independent")

alt.vconcat(top_graf, flop_graf).configure_view(stroke='transparent')

alt.VConcatChart(...)

In [29]:
podil_zanru("pohádky")

pocet,rok,len_right,podil,zanr
u32,i64,u32,f64,str
6,1989,3115,0.001926,"""pohádky"""
10,1990,3220,0.003106,"""pohádky"""
15,1991,3598,0.004169,"""pohádky"""
52,1992,4452,0.01168,"""pohádky"""
58,1993,5949,0.00975,"""pohádky"""
42,1994,6067,0.006923,"""pohádky"""
56,1995,6784,0.008255,"""pohádky"""
38,1996,7014,0.005418,"""pohádky"""
68,1997,7395,0.009195,"""pohádky"""


In [30]:
podil_zanru("erotick")

pocet,rok,len_right,podil,zanr
u32,i64,u32,f64,str
null,1989,3115,null,"""erotick"""
9,1990,3220,0.002795,"""erotick"""
9,1991,3598,0.002501,"""erotick"""
4,1992,4452,0.000898,"""erotick"""
6,1993,5949,0.001009,"""erotick"""
9,1994,6067,0.001483,"""erotick"""
null,1995,6784,null,"""erotick"""
10,1996,7014,0.001426,"""erotick"""
14,1997,7395,0.001893,"""erotick"""


In [31]:
podil_zanru("dívčí romány")

pocet,rok,len_right,podil,zanr
u32,i64,u32,f64,str
2,1989,3115,0.000642,"""dívčí romány"""
1,1990,3220,0.000311,"""dívčí romány"""
4,1991,3598,0.001112,"""dívčí romány"""
2,1992,4452,0.000449,"""dívčí romány"""
1,1993,5949,0.000168,"""dívčí romány"""
null,1994,6067,null,"""dívčí romány"""
null,1995,6784,null,"""dívčí romány"""
null,1996,7014,null,"""dívčí romány"""
1,1997,7395,0.000135,"""dívčí romány"""


In [32]:
podil_zanru("populárně-naučné publikace")

pocet,rok,len_right,podil,zanr
u32,i64,u32,f64,str
41,1989,3115,0.013162,"""populárně-naučné publikace"""
40,1990,3220,0.012422,"""populárně-naučné publikace"""
52,1991,3598,0.014452,"""populárně-naučné publikace"""
65,1992,4452,0.0146,"""populárně-naučné publikace"""
119,1993,5949,0.020003,"""populárně-naučné publikace"""
131,1994,6067,0.021592,"""populárně-naučné publikace"""
203,1995,6784,0.029923,"""populárně-naučné publikace"""
164,1996,7014,0.023382,"""populárně-naučné publikace"""
209,1997,7395,0.028262,"""populárně-naučné publikace"""


In [33]:
def grafik(z, funkce=podil_zanru):
    return alt.Chart(alt_friendly(funkce(z))).mark_line().encode(
        alt.X("rok:T",axis=alt.Axis(title=None)),
        alt.Y('podil:Q',axis=alt.Axis(
            labelExpr="datum.label * 100 + ' %'", orient='right', domainOpacity=0, tickColor='#DCDDD6',title=None
        ))
    ).configure_view(stroke='transparent')

In [34]:
grafik('erotické')

alt.Chart(...)

In [35]:
grafik('CD-ROM')

alt.Chart(...)

In [36]:
grafik("dvojjazyčná vydání")

alt.Chart(...)

In [37]:
grafik("fantasy romány"	)

alt.Chart(...)

In [38]:
grafik("učebnice základních škol")

alt.Chart(...)

In [39]:
grafik("aforismy")

alt.Chart(...)

In [40]:
grafik("rusk")

alt.Chart(...)

In [41]:
grafik("young adult")

alt.Chart(...)

In [42]:
grafik("příručky")

alt.Chart(...)

In [43]:
grafik("dívčí romány")

alt.Chart(...)

In [44]:
grafik("komiks")

alt.Chart(...)

In [45]:
grafik("detektiv")

alt.Chart(...)

In [46]:
grafik("rozhovory")

alt.Chart(...)

In [47]:
grafik("deníky")

alt.Chart(...)

In [48]:
grafik("autobiogr")

alt.Chart(...)

In [49]:
grafik("encyclopedias")

alt.Chart(...)

In [50]:
grafik("učebnice vysokých škol")

alt.Chart(...)

In [51]:
grafik("kuchař")

alt.Chart(...)

## Kuchařky

In [53]:
kucharky = df.explode("655_a").filter(
        pl.col("655_a").str.contains("kuchař")
    )

In [54]:
kucharky.sample(10)

100_ind1,100_a,100_7,100_4,100_d,100_q,100_c,100_b,100_e,001,leader,008,020_q,020_c,020_a,020_z,022_a,022_y,022_z,022_ind1,022_l,245_ind1,245_ind2,245_a,245_b,245_c,245_n,245_p,245_h,245_f,245_s,300_a,300_b,300_c,300_e,300_f,300_3,…,653_a,655_ind2,655_a,655_7,655_2,655_ind1,655_y,655_z,700_ind1,700_a,700_4,700_d,700_7,700_t,700_q,700_l,700_ind2,700_c,700_b,700_i,700_m,700_k,700_n,700_r,700_p,700_o,700_s,700_j,700_x,700_e,700_f,700_5,700_9,700_g,rok,stran,vazba
str,str,str,list[str],str,str,list[str],str,str,str,str,str,str,list[str],list[str],list[str],str,list[str],list[str],str,str,str,str,str,str,str,list[str],str,str,str,str,list[str],list[str],list[str],list[str],str,str,…,list[str],list[str],str,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],i64,i64,str
"""1""","""Mahelová, Lenka""","""jn20001227253""","[""aut""]",null,null,null,null,null,"""cpk20041448701""",""" cam a22 a 4500""","""041026s2004 xr af e f 0…","""(váz.) :""","[""Kč 249,00""]","[""80-86681-48-3""]",null,null,null,null,null,null,"""1""","""0""","""Zvěřina v naší kuchyni""","""342 receptů /""","""Lenka Mahelová""",null,null,null,null,null,"[""230 s., [8] s. barev. obr. příl. :""]","[""barev. il. ;""]","[""25 cm""]",null,null,null,…,null,"[""7"", ""9""]","""kuchařské recepty""","[""fd132687"", null]","[""czenas"", ""eczenas""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2004,230,"""pevná"""
"""1""","""Martin, Pavel""","""jn20001103238""","[""aut""]",null,null,null,null,null,"""cpk19980298145""",""" nam a22 a 4500""","""980204s1997 xr e 0…","""(brož.) :""","[""Kč 69,00""]","[""80-240-0059-8""]",null,null,null,null,null,null,"""1""","""0""","""Těstoviny""","""nejchutnější speciality na 150…","""Pavel Martin""",null,null,null,null,null,"[""107 s. ;""]",null,"[""20 cm""]",null,null,null,…,null,"[""7""]","""kuchařské recepty""","[""fd132687""]","[""czenas""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1997,107,"""brožovaná"""
"""1""","""Ambrová, Marika""","""mzk2003188765""","[""aut""]",null,null,null,null,null,"""cpk20031255824""",""" nam a22 a 4500""","""030908s2003 xr af e f 0…","""(brož.) :""","[""Kč 60,00""]","[""80-7334-020-8""]",null,null,null,null,null,null,"""1""","""0""","""Co chutná v horkých dnech""",null,"""Marika Ambrová""",null,null,null,null,null,"[""61 s., [8] s. obr. příl. :""]","[""barev. il. ;""]","[""21 cm""]",null,null,null,…,null,"[""7"", ""9""]","""kuchařské recepty""","[""fd132687"", null]","[""czenas"", ""eczenas""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2003,61,"""brožovaná"""
"""1""","""Ekart, Vladimír,""","""osa20181010441""","[""aut""]","""1971-""",null,null,null,null,"""nkc20183008835""",""" cam a22 i 4500""","""180920s2018 xr a e 0…","""(vázáno)""",null,"[""978-80-88284-04-8""]",null,null,null,null,null,null,"""1""","""0""","""To mám rád já""","""kuchařka s příběhem /""","""Vladimír Ekart""",null,null,null,null,null,"[""120 stran :""]","[""barevné ilustrace ;""]","[""28 cm""]",null,null,null,…,null,"[""7"", ""7"", … ""9""]","""kuchařské recepty""","[""fd132687"", ""fd131854"", … null]","[""czenas"", ""czenas"", … ""eczenas""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2018,120,"""pevná"""
"""1""","""Villa, Mariagrazia""","""osa2012732385""","[""aut""]",null,null,null,null,null,"""nkc20142648063""",""" cam a22 a 4500""","""141030s2014 xr a e f 0…","""(váz.)""",null,"[""

In [55]:
def kuchyne(slovo):
    return kucharky.filter(
        pl.col("245_a").str.contains("(?i)" + slovo) | pl.col("245_a").str.contains("(?i)" + slovo)
    ).group_by(
        "rok"
    ).len(
    ).join(kucharky.group_by("rok").len(), on="rok", how="right").with_columns((pl.col("len") / pl.col("len_right")).alias("podil")).sort(by="rok").rename({'len':'pocet'}).fill_null(0)

In [56]:
kuchyne('barbe')

pocet,rok,len_right,podil
u32,i64,u32,f64
0,1989,16,0.0
0,1990,20,0.0
0,1991,39,0.0
0,1992,57,0.0
0,1993,78,0.0
0,1994,80,0.0
0,1995,94,0.0
0,1996,91,0.0
0,1997,95,0.0


In [57]:
grafik('barbe', funkce=kuchyne)

alt.Chart(...)

In [58]:
grafik('vegan', funkce=kuchyne)

alt.Chart(...)

In [59]:
grafik('sous ', funkce=kuchyne)

alt.Chart(...)

In [60]:
grafik('svačin', funkce=kuchyne)

alt.Chart(...)

In [61]:
grafik('pečiv', funkce=kuchyne)

alt.Chart(...)

In [62]:
grafik('chleb', funkce=kuchyne)

alt.Chart(...)

In [63]:
grafik('peče', funkce=kuchyne)

alt.Chart(...)

In [64]:
grafik('diabet', funkce=kuchyne)

alt.Chart(...)

In [65]:
grafik('gril', funkce=kuchyne)

alt.Chart(...)

In [66]:
grafik('cukrov', funkce=kuchyne)

alt.Chart(...)

In [67]:
grafik('zdrav', funkce=kuchyne)

alt.Chart(...)

In [68]:
grafik('bez ', funkce=kuchyne)

alt.Chart(...)

In [69]:
kucharky.filter(pl.col("245_a").str.contains("pro")).with_columns(pl.col("245_a").map_elements(lambda x: x.split("pro")[1].strip())).group_by('245_a').len().sort(by='len',descending=True)

245_a,len
str,u32
"""diabetiky""",25
"""děti""",14
"""každou příležitost""",12
"""každého""",10
"""labužníky""",9
"""moderní ženu""",8
"""každý den""",8
"""začátečníky""",8
"""cukrářskou výrobu""",8


In [70]:
kucharky.filter(pl.col("245_a").str.contains(" s ")).with_columns(pl.col("245_a").map_elements(lambda x: x.split(" s ")[1].split(" ")[0].strip())).group_by('245_a').len().sort(by='len',descending=True)

245_a,len
str,u32
"""láskou""",10
"""Měsícem""",6
"""Ellou""",5
"""fantazií""",4
"""dětmi""",4
"""bylinkami""",4
"""Habadějem""",4
"""konopím""",4
"""pivem""",3


In [71]:
kucharky.filter(pl.col("245_a").str.contains(" bez ")).with_columns(pl.col("245_a").map_elements(lambda x: x.split(" bez ")[1].split(" ")[0].strip())).group_by('245_a').len().sort(by='len',descending=True)

245_a,len
str,u32
"""lepku""",24
"""cukru""",6
"""cholesterolu""",6
"""mléka""",6
"""vážení""",5
"""hladovění""",3
"""zkušeností""",2
"""servítků""",2
"""soli""",2


In [72]:
grafik(' s ', funkce=kuchyne)

alt.Chart(...)

In [73]:
grafik(' bez ', funkce=kuchyne)

alt.Chart(...)

In [74]:
grafik('frit', funkce=kuchyne)

alt.Chart(...)

In [75]:
grafik('indi', funkce=kuchyne)

alt.Chart(...)

In [76]:
grafik('japon', funkce=kuchyne)

alt.Chart(...)

In [77]:
grafik('vietn', funkce=kuchyne)

alt.Chart(...)

In [78]:
grafik('pán[ev]', funkce=kuchyne)

alt.Chart(...)

In [79]:
grafik('mikrov', funkce=kuchyne)

alt.Chart(...)

In [80]:
grafik('gril', funkce=kuchyne)

alt.Chart(...)

In [81]:
grafik('sex', funkce=kuchyne)

alt.Chart(...)

In [82]:
grafik('babi', funkce=kuchyne)

alt.Chart(...)

In [83]:
grafik('pomaz', funkce=kuchyne)

alt.Chart(...)

In [84]:
grafik('cukr', funkce=kuchyne)

alt.Chart(...)

In [85]:
grafik('hrní', funkce=kuchyne)

alt.Chart(...)

In [86]:
grafik('hubn', funkce=kuchyne)

alt.Chart(...)

In [87]:
grafik('omáč', funkce=kuchyne)

alt.Chart(...)

In [88]:
grafik('pol[íé]v', funkce=kuchyne)

alt.Chart(...)

In [89]:
grafik('pomaz', funkce=kuchyne)

alt.Chart(...)

In [90]:
grafik('keto', funkce=kuchyne)

alt.Chart(...)

In [91]:
grafik('paleo', funkce=kuchyne)

alt.Chart(...)

In [92]:
grafik('houb', funkce=kuchyne)

alt.Chart(...)

In [93]:
grafik('(kvas|kvaš|ferment)', funkce=kuchyne)

alt.Chart(...)

In [94]:
grafik('makrobiot', funkce=kuchyne)

alt.Chart(...)

In [95]:
grafik('bílkov', funkce=kuchyne)

alt.Chart(...)

In [96]:
kucharky.select(pl.col("245_a")).to_series().to_list()

['Kuchařka rodiny dona Corleona',
 'Lávové koláčky',
 'Italská kuchařka',
 'Biblická hostina',
 'Bravo, šéfe!',
 'Rodinná kuchařka',
 'Kuchařka pro zahradní pohoštění',
 'Pomazánky, saláty doma i na chalupě',
 'Skleničková kuchařka',
 'Kuchařka babičky Chadimové',
 'Chilli',
 'Velká italská kuchařka',
 'Moučníky sestry Anastázie',
 'Kuchařka',
 'Asijská kuchyně po česku',
 'Zdravé pečení',
 'Paleo dieta',
 'Diety při onemocnění žlučníku',
 'Kluci v akci 3',
 'Zázračné semínko chia',
 'Moučníky, dezerty a pokrmy z těstovin a rýže',
 'Hormony štěstí',
 'Co se dříve jedlo pod štramberskou trúbou',
 'Nejlepší recepty sladké hrníčkové kuchyně',
 'Tajemství židovské kuchyně, aneb, 270 tradičních židovských receptů',
 'Nejlepší recepty',
 'Dieta bez diety, aneb, Brambory v kuchyni pro štíhlou linii',
 'České vánoce',
 'Grilování doma i na chatě',
 'Od artyčoku po zelí',
 'Aby dětem chutnalo...',
 'Rychlodiety',
 'Vaříme z české zahrádky',
 'Suši bar',
 'Pečeme chleba',
 'S Hubertem na cestách